## Workshop on Scientific Computing in Economics with Python and Julia

## Monday Afternoon Session: Julia for Economists

**Chase Coleman**

## What makes julia the right tool for what we do?

* Fast
* Flexible
* Intuitive
* Quickly growing package ecosystem (We don't want to re-invent the wheel)

## Distributions.jl

A Julia package for probability distributions and associated functions. Particularly, Distributions implements:

* Moments (e.g mean, variance, skewness, and kurtosis), entropy, and other properties
* Probability density/mass functions (pdf) and their logarithm (logpdf)
* Moment generating functions and characteristic functions
* Sampling from population or from a distribution
* Maximum likelihood estimation

In particular, it provides a very natural (and consistent!) way to work with about 70 different distributions.

In [1]:
using Distributions

We will create a standard normal distribution and do a variety of things with it below.

In [8]:
# Create the standard normal distribution
nrv = Normal(0.0, 1.0)

# Get the 1st and 99th percentile
nrv_quantiles = quantile(nrv, [0.01, 0.99])

# Get the value of pdf (and cdf) of the 1st and 99th percentile
nrv_pdf = pdf(nrv, nrv_quantiles)
nrv_cdf = cdf(nrv, nrv_quantiles)

# Sample from the distribution (samples 1 draw from distribution unless specified)
rand(nrv)
rand(nrv, 10)

10-element Array{Float64,1}:
  0.805947  
 -0.0746159 
 -0.189863  
 -0.00602921
  0.576053  
 -0.146408  
  0.628677  
  0.48154   
 -0.715621  
 -0.695809  

In [28]:
typeof(nrv)

Distributions.Normal

Notice that we can use almost exactly the same code -- We only change the distribution name -- to get these same items from the inverse gamma distribution

In [26]:
ivg = InverseGamma(7.0, 4.0)

# Get the 1st and 99th percentile
ivg_quantiles = quantile(ivg, [0.01, 0.99])

# Get the value of pdf (and cdf) of the 1st and 99th percentile
ivg_pdf = pdf(ivg, ivg_quantiles)
ivg_cdf = cdf(ivg, ivg_quantiles)

# Sample from the distribution (samples 1 draw from distribution unless specified)
rand(ivg)
rand(ivg, 10)

10-element Array{Float64,1}:
 1.03085 
 0.412152
 0.581367
 0.784663
 0.667347
 0.376235
 0.582554
 0.717137
 0.673634
 1.62976 

In [27]:
[mean(nrv) mean(ivg)
 std(nrv) std(ivg)
 skewness(nrv) skewness(ivg)
 kurtosis(nrv) kurtosis(ivg)]

4x2 Array{Float64,2}:
 0.0   0.666667
 1.0   0.298142
 0.0   2.23607 
 0.0  12.0     

## Interpolations.jl

This package implements a variety of interpolation schemes for the Julia langauge. It has the goals of ease-of-use, broad algorithmic support, and exceptional performance.

This package is still relatively new. Currently its support is best for B-splines and also supports irregular grids.

In [30]:
using Interpolations

The first thing we need to do is to create an interpolator -- By default this will be defined on [1, Npts]. We need to specify several things:

* Data
* Type of interpolation
  - Order of interpolation
  - Boundary Conditoins
* Where data is located

In [61]:
lb, ub = 0.1, 10.0
x = linspace(lb, ub, 25)
u = log(x)

# Create a BSpline using linear interpolation with data on the grid
lin_itp = interpolate(u, BSpline(Linear()), OnGrid())
qua_itp = interpolate(u, BSpline(Quadratic(Natural())), OnGrid());

We can evaluate the interpolator by using "indexing"

In [62]:
println("Log(x) is: ", round(log(lb), 4))
println("Interpolated value at x is: ", round(lin_itp[lb], 4))

Log(x) is: -2.3026
Interpolated value at x is: -3.7733


What happened??? This is on a grid point, why doesn't it match. This is what I meant when I said it would be defined by default on [1, Npts], we really need to give it the index of the point.

In [63]:
println("Log(x) is: ", round(log(x[1]), 4))
println("Interpolated value at x is: ", round(lin_itp[1], 4))

Log(x) is: -2.3026
Interpolated value at x is: -2.3026


It could get annoying to always be working in indexes for us, but there is a "scaling" feature that allows us to move the domain to where we would like to work.

In [64]:
lin_itp_s = scale(lin_itp, x)
qua_itp_s = scale(qua_itp, x);

In [65]:
println("Log(x) is: ", round(log(lb), 4))
println("Interpolated value at x is: ", round(lin_itp_s[lb], 4))

Log(x) is: -2.3026
Interpolated value at x is: -2.3026


Extrapolation is by default set to linearly extrapolate outside of the bounds, but you can ask it to do

* `Throw` : Throw an error if asked to extrapolate
* `Flat` : Evaluate anything outside grid to the closest grid point
* `Linear` : Linear extrapolate outside of bounds

In [67]:
println(extrapolate(lin_itp_s, Interpolations.Throw())[lb + 0.05])
println(extrapolate(lin_itp_s, Interpolations.Throw())[lb - 0.05])

-2.104508665718352


LoadError: LoadError: BoundsError
while loading In[67], in expression starting on line 2

Best of all, `Interpolations.jl` is _very_ fast.

In [108]:
n = 10_000_000
fine_grid = linspace(lb, ub, n)

function fill_values(itp::ScaledInterpolation, x::LinSpace)

    # Get number of values and allocate space
    n = length(x)
    interp_values = Array(Float64, n)

    # Fill values
    for i=1:n
        interp_values[i] = itp[x[i]]
    end

    return interp_values
end

@time fill_values(lin_itp_s, fine_grid);

  0.204638 seconds (4.55 k allocations: 76.505 MB, 3.02% gc time)


## NLsolve.jl

In [110]:
using NLsolve

Imagine solving the following system of non-linear equations:

$$\bar{

## Optim.jl

In [111]:
using Optim

## QuantEcon.jl

General purpose tools for 

In [112]:
using QuantEcon

## Motivating Example